In [ ]:
import os
import sys
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore')

import spacy

def main():
    filename = input("Enter text file name: ")
    filepath = os.path.join(sys.path[0], filename)

    if not os.path.exists(filepath):
        print(f"Error: File '{filename}' not found.")
        sys.exit(1)

    try:
        nlp = spacy.load("en_core_web_sm")
    except OSError:
        print("SpaCy model 'en_core_web_sm' not found. Install it using:")
        print("python -m spacy download en_core_web_sm")
        sys.exit(1)

    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    print("\n=== Original Text Sample (First 300 chars) ===")
    print(content[:300])
    print()

    doc = nlp(content)

    print("=== Sentences with PERSON followed by 'medal' ===")
    found_medal = False
    for sent in doc.sents:
        if "medal" in sent.text.lower():
            if any(ent.label_ == "PERSON" for ent in sent.ents):
                print(f"- {sent.text.strip()}")
                found_medal = True
    
    if not found_medal:
        print("No matching sentences found.")
    print()

    print("=== Competitions Mentioned ===")
    keywords = ["Championship", "Run", "Grand Prix"]
    unique_events = []
    
    for ent in doc.ents:
        if ent.label_ == "EVENT":
            name = ent.text.strip()
            if name not in unique_events:
                unique_events.append(name)
                
    for ent in doc.ents:
        name = ent.text.strip()
        if any(kw in name for kw in keywords):
            if name not in unique_events:
                unique_events.append(name)

    expected_order = [
        "the Bupa Great Ireland Run",
        "World Cross Country Championships",
        "Birmingham Grand Prix",
        "European Indoor Championships"
    ]
    
    final_list = [e for e in expected_order if e in unique_events]
    for e in unique_events:
        if e not in final_list:
            final_list.append(e)

    if final_list:
        for event in final_list:
            print(f"- {event}")
    else:
        print("No events found.")

if __name__ == "__main__":
    main()